In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import torch
from UnarySim.sw.quantum.linear_complex import UnaryLinearComplex, LinearComplex
from UnarySim.sw.bitstream.gen import RNG, SourceGen, BSGen
from UnarySim.sw.metric.metric import ProgressiveError
import matplotlib.pyplot as plt
import time

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
def unary_linear_complex_test(rng="Sobol", in_feature=16, out_feature=16, bitwidth=8):
    mode = "bipolar"
    bstype = torch.float
    buftype = torch.float
    randtype = torch.float
    scaled = [True]
    result_pe_r = []
    result_pe_i = []
    
    for scale in scaled:
        length = 2**bitwidth
        result_pe_cycle_r = []
        result_pe_cycle_i = []

        wr = torch.rand(out_feature, in_feature).mul(2).sub(1).mul(length).round().div(length).to(device)
        wi = torch.rand(out_feature, in_feature).mul(2).sub(1).mul(length).round().div(length).to(device)
#         wr = (torch.tensor([[3., 4.], [5., 6.]])).t().to(device)
#         wi = (torch.tensor([[3., 4.], [5., 6.]])).t().to(device)
#         print("wr:", wr)
#         print("wi:", wi)
        fc = LinearComplex(in_feature, out_feature, wr, wi).to(device)
        
        ufc = UnaryLinearComplex(in_feature, out_feature, wr, wi, 
                                 bitwidth=bitwidth, scaled=scale, 
                                 bstype=bstype, buftype=buftype, randtype=randtype).to(device)

        iVec_r = ((torch.rand(1, in_feature).mul(2).sub(1)*length).round()/length).to(device)
        iVec_i = ((torch.rand(1, in_feature).mul(2).sub(1)*length).round()/length).to(device)
#         iVec_r = (torch.tensor([[1., 2.]])).to(device)
#         iVec_i = (torch.tensor([[1., 2.]])).to(device)
#         print("iVec_r:", iVec_r)
#         print("iVec_i:", iVec_i)
        
        oVec_r, oVec_i = fc(iVec_r, iVec_i)
        
        if scale is True:
            oVec_r = oVec_r.mul(length).floor()/length
            oVec_i = oVec_i.mul(length).floor()/length
        else:
            oVec_r = oVec_r.clamp(-1.,1.).mul(length).floor()/length
            oVec_i = oVec_i.clamp(-1.,1.).mul(length).floor()/length
        
        iVecSRC_r = SourceGen(iVec_r, bitwidth=bitwidth, mode=mode, randtype=randtype)().to(device)
        iVecSRC_i = SourceGen(iVec_i, bitwidth=bitwidth, mode=mode, randtype=randtype)().to(device)

        iVecRNG = RNG(bitwidth, 1, rng, randtype=randtype)().to(device)

        iVecBS_r = BSGen(iVecSRC_r, iVecRNG, bstype=bstype).to(device)
        iVecBS_i = BSGen(iVecSRC_i, iVecRNG, bstype=bstype).to(device)

        iVecPE_r = ProgressiveError(iVec_r, mode=mode).to(device)
        iVecPE_i = ProgressiveError(iVec_i, mode=mode).to(device)
        
        oVecPE_r = ProgressiveError(oVec_r, mode=mode).to(device)
        oVecPE_i = ProgressiveError(oVec_i, mode=mode).to(device)
        
        with torch.no_grad():
            idx = torch.zeros(iVecSRC_r.size()).type(torch.long).to(device)
            start_time = time.time()
            for i in range(length):
                iBS_r = iVecBS_r(idx + i)
                iVecPE_r.Monitor(iBS_r)
                iBS_i = iVecBS_i(idx + i)
                iVecPE_i.Monitor(iBS_i)

                oVecU_r, oVecU_i = ufc(iBS_r, iBS_i)
                oVecPE_r.Monitor(oVecU_r)
                oVecPE_i.Monitor(oVecU_i)
                if scale is True:
                    err_r = oVecPE_r()[0]*(in_feature*2)-oVec_r
                    rmse_r = 1-torch.sqrt(torch.mean(err_r**2)).item()/out_feature/2
                    err_i = oVecPE_i()[0]*(in_feature*2)-oVec_i
                    rmse_i = 1-torch.sqrt(torch.mean(err_i**2)).item()/out_feature/2
                else:
                    err_r = oVecPE_r()[1]
                    rmse_r = 1-torch.sqrt(torch.mean(err_r**2)).item()
                    err_i = oVecPE_i()[1]
                    rmse_i = 1-torch.sqrt(torch.mean(err_i**2)).item()
                result_pe_cycle_r.append(rmse_r)
                result_pe_cycle_i.append(rmse_i)
            print("--- %s seconds ---" % (time.time() - start_time))
            print("input r error: ", "min:", torch.min(iVecPE_r()[1]).item(), "max:", torch.max(iVecPE_r()[1]).item())
            if scale is True:
                print("output r error: ", 
                      "min:", torch.min(err_r).item()/out_feature/2, 
                      "max:", torch.max(err_i).item()/out_feature/2)
                result_pe_r = (err_r[0]).cpu().numpy()/out_feature/2
            else:
                print("output r error: ", "min:", torch.min(oVecPE_r()[1]).item(), "max:", torch.max(oVecPE_r()[1]).item())
                result_pe_r = oVecPE_r()[1][0].cpu().numpy()
            print("input i error: ", "min:", torch.min(iVecPE_i()[1]).item(), "max:", torch.max(iVecPE_i()[1]).item())
            if scale is True:
                print("output i error: ", 
                      "min:", torch.min(oVecPE_i()[0]*(in_feature*2)-oVec_i).item()/out_feature/2, 
                      "max:", torch.max(oVecPE_i()[0]*(in_feature*2)-oVec_i).item()/out_feature/2)
                result_pe_i = ((oVecPE_i()[0]*(in_feature*2)-oVec_i)[0]).cpu().numpy()/out_feature/2
            else:
                print("output i error: ", "min:", torch.min(oVecPE_i()[1]).item(), "max:", torch.max(oVecPE_i()[1]).item())
                result_pe_i = oVecPE_i()[1][0].cpu().numpy()
            print("r error distribution=========>")
            plt.figure(figsize=(3,1.5))
            fig = plt.hist(result_pe_r, bins='auto')  # arguments are passed to np.histogram
            plt.title("data: "+mode+", scaled: "+str(scale))
            plt.show()
            print("r progressive accuracy=========>")
            plt.figure(figsize=(3,1.5))
            fig = plt.plot(result_pe_cycle_r)  # arguments are passed to np.histogram
            plt.title("data: "+mode+", scaled: "+str(scale))
            plt.show()
            
            print("i error distribution=========>")
            plt.figure(figsize=(3,1.5))
            fig = plt.hist(result_pe_i, bins='auto')  # arguments are passed to np.histogram
            plt.title("data: "+mode+", scaled: "+str(scale))
            plt.show()
            print("i progressive accuracy=========>")
            plt.figure(figsize=(3,1.5))
            fig = plt.plot(result_pe_cycle_i)  # arguments are passed to np.histogram
            plt.title("data: "+mode+", scaled: "+str(scale))
            plt.show()


In [26]:
rng = "Sobol"
in_feature = 16
out_feature = in_feature
bitwidth = 8
unary_linear_complex_test(rng, in_feature, out_feature, bitwidth)

TypeError: super(type, obj): obj must be an instance or subtype of type